In [1]:
import os
# Hide GPU from visible devices
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
# Verify it only sees the CPU
print("Devices detected:", tf.config.list_physical_devices())

2026-02-07 07:02:10.365741: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-07 07:02:10.397632: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-07 07:02:11.656395: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Devices detected: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2026-02-07 07:02:12.931256: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
# import the library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
# input the data
movies = pd.read_csv("ml-latest-small/movies.csv")
users = pd.read_csv("ml-latest-small/ratings.csv")

In [5]:
# see and understand the data features
#movies.info()
#users.info()
print(movies.head(3))
print(users.head(3))

   movieId                    title  \
0        1         Toy Story (1995)   
1        2           Jumanji (1995)   
2        3  Grumpier Old Men (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224


In [6]:
#merge the 2 data sets to get related features
data = pd.merge(
    users,
    movies,
    on="movieId",
    how="left"
)

In [7]:
data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [8]:
# derive two data sets for two towers and target values
X_userId = data["userId"].values
X_movieId = data["movieId"].values
X_genres = data["genres"].str.get_dummies(sep='|').values # encode the genres using one-hot encode
y_rating = data["rating"].values

# split the data into train and test
(X_uid_train, X_uid_test, 
 X_mid_train, X_mid_test, 
 X_gen_train, X_gen_test, 
 y_train, y_test) = train_test_split(
    X_userId, X_movieId, X_genres, y_rating, 
    test_size=0.2, 
    random_state=42
)

In [14]:
# build the two towers using tensorflow
import tensorflow as tf
tf.keras.backend.clear_session()

In [15]:

# user tower to generate user vector
num_user =X_uid_train.max() + 1 # for the Embedding stage
user_id_input = tf.keras.layers.Input(shape=(1,), name="user_id")
user_emb = tf.keras.layers.Embedding(num_user, 32)(user_id_input)
user_vec = tf.keras.layers.Flatten()(user_emb)
user_vec = tf.keras.layers.Dense(64, activation="relu")(user_vec)
user_output = tf.keras.layers.Dense(32, activation="linear")(user_vec)

user_tower_model = tf.keras.Model(inputs=user_id_input, outputs=user_output)

# movie tower to generate movie vector
num_movie = X_mid_train.max() + 1
movie_id_input = tf.keras.layers.Input(shape=(1,), name="movie_id")
movie_genre_input = tf.keras.layers.Input(shape=(20,), name="movie_genre")
movie_id_emb = tf.keras.layers.Embedding(num_movie, 32)(movie_id_input)# embed the movie id
movie_id_vec = tf.keras.layers.Flatten()(movie_id_emb)
merge_data = tf.keras.layers.concatenate([movie_id_vec, movie_genre_input])
movie_vec = tf.keras.layers.Dense(64, activation="relu")(merge_data)
movie_output = tf.keras.layers.Dense(32, activation="linear")(movie_vec)

movie_tower_model = tf.keras.Model(inputs=[movie_id_input, movie_genre_input], outputs=movie_output)

In [16]:
# we create the vectors for both user and movie
user_vector = user_tower_model(user_id_input)
movie_vector = movie_tower_model([movie_id_input, movie_genre_input])

# calculate the dot product (similarity)
dot_product = tf.keras.layers.Dot(axes=1)([user_vector, movie_vector])

# construct the final model 
model = tf.keras.Model(
    inputs=[user_id_input, movie_id_input, movie_genre_input],
    outputs=dot_product
)
model.compile(optimizer="Adam", loss="mse")

In [17]:
# trian the model
model.fit(
    x = [
        X_uid_train,
        X_mid_train,
        X_gen_train
        
    ],

    y = y_train,
    epochs=10,
    batch_size=32
)

Epoch 1/10
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - loss: 0.9747
Epoch 2/10
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - loss: 0.7084
Epoch 3/10
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - loss: 0.6380
Epoch 4/10
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - loss: 0.5757
Epoch 5/10
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - loss: 0.5194
Epoch 6/10
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - loss: 0.4721
Epoch 7/10
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - loss: 0.4318
Epoch 8/10
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - loss: 0.3953
Epoch 9/10
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - loss: 0.3679
Epoch 10/10
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - loss: 0.3446


In [20]:
# test the model
loss = model.evaluate (
    x = [
        X_uid_test,
        X_mid_test,
        X_gen_test
    ],
    y = y_test
)
print(f"Final Test MSE: {loss:.4f}")
print(f"RMSE: {loss**0.5:.4f}")

631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 827us/step - loss: 0.8451
Final Test MSE: 0.8451
RMSE: 0.9193


In [ ]:
# Top K-Recommendation test using Retrieval and Ranking method